In [13]:
import pandas as pd
import dill
import plotly.express as px
pd.options.plotting.backend = "plotly"

In [8]:
diffs = pd.read_pickle('diff_from_schedule.pkl')

In [25]:
# trips that were delayed at least once at a station
all_delayed = diffs.apply(lambda c: (c.dt.total_seconds() > 20), axis=0).any(axis=1).sum()

In [28]:
# count trips that get delayed but end up at 37 on time
only_at_end = (diffs['37'].dt.total_seconds() > 20).sum()
# can see from this that most trips recover themselves by speeding up
# so the 'terminal time' is not a good measure of lateness
(only_at_end, all_delayed)

(88, 206)

In [18]:
barchart_data = pd.concat([
pd.DataFrame(data={'type': 'delayed',
                   'count':diffs.apply(lambda c: (c.dt.total_seconds() > 0).sum(), axis=0)
                   }),
    pd.DataFrame(data=
                 {'type': 'not delayed',
                   'count':diffs.apply(lambda c: (c.dt.total_seconds() < 0).sum(), axis=0)
                   })])
barchart_data['station'] = barchart_data.index
#px.bar(barchart_data, x='station', y='count', color='type', title='Delays by Station')

In [ ]:
# not sure I care about plotting the actual delayed trips
fig = go.Figure(data=[
    go.Scatter(mode='lines+markers',
name='Delay',
y=(late_trip - (onetime_trip + (late_trip[1] - onetime_trip[1])) ) + pd.to_datetime('2020-08-04'),
x=arrivals_and_expected.columns)])

# same as taking difference of trip lengths?
# I think so I'm just not seeing it
# (E_1 - S_1) - (E_2 - S_2)
# (E_2 - E_1) - (S_2 - S_1)
fig.add_trace(go.Scatter(mode='lines+markers',
name='Derivative',
y=(late_trip - (onetime_trip + (late_trip[1] - onetime_trip[1])) ).diff()+ pd.to_datetime('2020-08-04'),
x=arrivals_and_expected.columns))

fig.update_layout(
    title=f'Delays',
    xaxis_title=f'Station',
    yaxis_title=f'Difference'
)